In [1]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
from PIL import ImageFont, ImageDraw, Image
import tensorflow as tf
from tensorflow.python.framework.ops import EagerTensor
from tensorflow.keras import layers, Model
import cv2
from tensorflow.keras.models import load_model

%matplotlib inline

C:\Users\user\AppData\Local\Temp\ipykernel_9288\3015848435.py:6: DeprecationWarning: scipy.misc is deprecated and will be removed in 2.0.0
  import scipy.misc


In [2]:


def yolo_filter_boxes(boxes, conf, probs, thresh=0.6):
    """Filters YOLO boxes based on confidence and class scores."""
    
    # Compute box scores
    scores = probs * conf

    # Get class with highest score & corresponding score
    classes = tf.argmax(scores, axis=-1)
    class_scores = tf.reduce_max(scores, axis=-1)

    # Create mask for filtering
    mask = class_scores >= thresh

    # Apply mask
    return tf.boolean_mask(class_scores, mask), tf.boolean_mask(boxes, mask), tf.boolean_mask(classes, mask)


In [3]:
def iou(b1, b2):
    """Compute the Intersection over Union (IoU) between two boxes."""
    
    x1, y1, x2, y2 = b1
    x1_, y1_, x2_, y2_ = b2

    # Compute intersection
    xi1, yi1 = max(x1, x1_), max(y1, y1_)
    xi2, yi2 = min(x2, x2_), min(y2, y2_)
    inter_w, inter_h = max(0, yi2 - yi1), max(0, xi2 - xi1)
    inter_area = inter_w * inter_h

    # Compute union
    area1 = (x2 - x1) * (y2 - y1)
    area2 = (x2_ - x1_) * (y2_ - y1_)
    union_area = area1 + area2 - inter_area

    return inter_area / union_area


In [4]:


def yolo_nms(scores, boxes, classes, max_boxes=10, iou_thresh=0.5):
    """Applies Non-Max Suppression (NMS) to filter boxes."""
    
    max_boxes_tensor = tf.Variable(max_boxes, dtype='int32')

    # Get indices of boxes to keep
    nms_idx = tf.image.non_max_suppression(boxes, scores, max_boxes_tensor, iou_thresh)

    # Select filtered scores, boxes, and classes
    return tf.gather(scores, nms_idx), tf.gather(boxes, nms_idx), tf.gather(classes, nms_idx)


In [5]:


def yolo_eval(outputs, img_shape=(720, 1280), max_boxes=10, score_thresh=0.6, iou_thresh=0.5):
    """Converts YOLO model output to final filtered boxes, scores, and classes."""
    
    box_xy, box_wh, box_conf, box_probs = outputs

    # Convert boxes to corner coordinates
    boxes = yolo_boxes_to_corners(box_xy, box_wh)

    # Apply score filtering
    scores, boxes, classes = yolo_filter_boxes(boxes, box_conf, box_probs, score_thresh)

    # Scale boxes to original image size
    boxes = scale_boxes(boxes, img_shape)

    # Apply non-max suppression
    return yolo_nms(scores, boxes, classes, max_boxes, iou_thresh)


In [6]:
import tensorflow as tf
import numpy as np

# Set both TensorFlow and NumPy seeds
tf.random.set_seed(1)
np.random.seed(1)

def yolo_boxes_to_corners(xy, wh):
    """Convert YOLO box format (x, y, w, h) to (x1, y1, x2, y2)"""
    min_corner = xy - (wh / 2)
    max_corner = xy + (wh / 2)
    return tf.concat([min_corner, max_corner], axis=-1)

def scale_boxes(boxes, img_shape):
    """Rescale the YOLO boxes to the original image size."""
    height, width = tf.cast(img_shape[0], tf.float32), tf.cast(img_shape[1], tf.float32)
    scale = tf.stack([width, height, width, height])
    return boxes * scale

# Generate YOLO outputs with fixed seed
yolo_outputs = (
    tf.random.normal([19, 19, 5, 2], mean=1, stddev=4),
    tf.random.normal([19, 19, 5, 2], mean=1, stddev=4),
    tf.random.normal([19, 19, 5, 1], mean=1, stddev=4),
    tf.random.normal([19, 19, 5, 80], mean=1, stddev=4)
)

# Run YOLO evaluation
scores, boxes, classes = yolo_eval(yolo_outputs)

# Print results
print("scores[2] =", scores[2].numpy())
print("boxes[2] =", boxes[2].numpy())
print("classes[2] =", classes[2].numpy())
print("scores.shape =", scores.numpy().shape)
print("boxes.shape =", boxes.numpy().shape)
print("classes.shape =", classes.numpy().shape)


scores[2] = 152.68243
boxes[2] = [ 1530.0006   -3760.4392     831.5534     -19.682007]
classes[2] = 25
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)


In [7]:

def darknet53():
    inputs = layers.Input(shape=(416, 416, 3))
    x = layers.Conv2D(32, (3,3), padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D(2,2)(x)
    x = layers.Conv2D(64, (3,3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(2,2)(x)
    x = layers.Conv2D(128, (3,3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(2,2)(x)
    x = layers.Conv2D(256, (3,3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(2,2)(x)
    x = layers.Conv2D(512, (3,3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(2,2)(x)
    return Model(inputs, x, name='Darknet53')

def yolo_head(x):
    x = layers.Conv2D(1024, (3,3), padding='same', activation='relu')(x)
    x = layers.Conv2D(5 + 80, (1,1), padding='same', activation='sigmoid')(x)  # Adjust output for classes
    return x

def yolo_model():
    inputs = layers.Input(shape=(416, 416, 3))
    darknet = darknet53()(inputs)
    outputs = yolo_head(darknet)
    return Model(inputs, outputs, name='YOLO')

# Create the model
model = yolo_model()
model.summary()


Model: "YOLO"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 416, 416, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Darknet53 (Functional)          │ (None, 13, 13, 512)    │     1,568,576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 13, 13, 1024)   │     4,719,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 13, 13, 85)     │        87,125 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,375,317 (24.32 MB)

 Trainable params: 6,375,317 (24.32 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (416, 416))
    image = image / 255.0  # Normalize to [0,1]
    return np.expand_dims(image, axis=0)

# --- 5️⃣ Load Dataset (Image & Labels) ---
import os
import cv2
import numpy as np

def preprocess_image(image_path):
    """Loads an image, resizes it, and normalizes it."""
    if not os.path.exists(image_path):
        print(f"Error: Image file '{image_path}' does not exist!")
        return None

    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not read image '{image_path}'. Skipping...")
        return None

    image = cv2.resize(image, (416, 416))
    image = image / 255.0  # Normalize to [0,1]
    return image

import os
import random

def load_data(image_dir, label_dir, subset_size=50):
    """ Load a reduced dataset with a subset of images and labels. """
    
    image_files = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]
    random.shuffle(image_files)  # Shuffle for randomness

    # Select only a subset of images
    image_files = image_files[:subset_size]

    images = []
    labels = []
    
    for img_file in image_files:
        img_path = os.path.join(image_dir, img_file)
        label_path = os.path.join(label_dir, img_file.replace('.jpg', '.txt'))
        
        # Read and preprocess image
        img = cv2.imread(img_path)
        img = cv2.resize(img, (320, 320))  # Resize to save memory
        img = img / 255.0  # Normalize
        
        images.append(img)

        # Read label file
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                label_data = f.readlines()
            labels.append(label_data)
        else:
            labels.append([])  # Empty label for missing files

    return images, labels






In [13]:
# **Load a smaller dataset**
images, labels = load_data(
    r"C:\Users\user\Downloads\archive\vehicle dataset\train\images",
    r"C:\Users\user\Downloads\archive\vehicle dataset\train\labels",
    subset_size=50  # Adjust this number as needed
)

In [14]:
import tensorflow as tf

def yolo_loss(y_true, y_pred):
    """
    Custom loss function for YOLO.
    """
    mse = tf.keras.losses.MeanSquaredError()
    ce = tf.keras.losses.CategoricalCrossentropy()

    # Extract values from predictions
    pred_xy = y_pred[..., :2]
    pred_wh = y_pred[..., 2:4]
    pred_conf = y_pred[..., 4:5]
    pred_class = y_pred[..., 5:]

    # Extract ground truth values
    true_xy = y_true[..., :2]
    true_wh = y_true[..., 2:4]
    true_conf = y_true[..., 4:5]
    true_class = y_true[..., 5:]

    # Compute losses
    bbox_loss = mse(true_xy, pred_xy) + mse(true_wh, pred_wh)
    conf_loss = mse(true_conf, pred_conf)
    class_loss = ce(true_class, pred_class)

    total_loss = bbox_loss + conf_loss + class_loss
    return total_loss


In [11]:
def data_generator(image_paths, labels, batch_size=16):
    while True:
        for i in range(0, len(image_paths), batch_size):
            batch_images = []
            batch_labels = []
            
            for j in range(i, min(i + batch_size, len(image_paths))):
                img_path = image_paths[j]
                
                # **DEBUGGING CHECK**
                if not isinstance(img_path, str):
                    print(f"Skipping invalid path at index {j}: {img_path}")
                    continue
                
                img = cv2.imread(img_path)  # Read image
                
                # **CHECK IF IMAGE LOADED CORRECTLY**
                if img is None:
                    print(f"Error loading image: {img_path}")
                    continue
                
                img = cv2.resize(img, (416, 416))
                img = img / 255.0  # Normalize
                batch_images.append(img)
                batch_labels.append(labels[j])
            
            if batch_images:  # Ensure batch is not empty
                yield np.array(batch_images, dtype=np.float32), np.array(batch_labels, dtype=np.float32)


In [12]:
batch_size = 16
train_gen = data_generator(images, labels, batch_size)

# Compile YOLO Model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train Model
history = model.fit(train_gen, steps_per_epoch=len(images) // batch_size, epochs=10)

# Save Model
model.save("yolo_trained_model.h5")

# Evaluate Model Performance
loss, accuracy = model.evaluate(train_gen, steps=len(images) // batch_size)
print(f"Final Accuracy: {accuracy * 100:.2f}%")


NameError: name 'images' is not defined